In [1]:
# import pandas as pd
import MagmaPandas as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import meltInc.plotting as p
from functools import partial
from MagmaPandas.geochemistry.fO2 import fO2_QFM
from MagmaPandas.geochemistry.Kd_ol_melt import Kd_FeMg_vectorised
from MagmaPandas.geochemistry.Fe_redox import Fe_redox
from MagmaPandas.geochemistry.PEC import Fe_equilibrate

p.layout()
config = mp.configuration()
config.print()

Kd Fe-Mg ol-melt.......................toplis
Melt Fe3+/Fe2+........................borisov
Melt thermometer...............putirka2008_14
Volatile solubility model......IaconoMarziano


In [2]:
melt = mp.read_melt_inclusion('./melt.csv', index_col=['name'], total_col='total')
inclusions = melt.iloc[0:10,:]

In [4]:
forsterite_host = 0.8
P_bar = 1e3

In [5]:
w, F1, oc = melt.Fe_equilibrate(forsterite_host, P_bar)


/Users/thomas/Desktop/MagmaPandas/src/MagmaPandas/geochemistry/fO2.py:163: UserWarning: O'Neill fO2: temperatures above 1420K present
  w.warn("O'Neill fO2: temperatures above 1420K present")


T correction 08

In [6]:
o_crystallised = []
for _, inclusion in melt.iterrows():
    *_, OC = Fe_equilibrate(inclusion, forsterite_host, P_bar)
    o_crystallised.append(OC)
o_crystallised

/Users/thomas/Desktop/MagmaPandas/src/MagmaPandas/geochemistry/fO2.py:163: UserWarning: O'Neill fO2: temperatures above 1420K present
  w.warn("O'Neill fO2: temperatures above 1420K present")
/Users/thomas/Desktop/MagmaPandas/src/MagmaPandas/geochemistry/fO2.py:163: UserWarning: O'Neill fO2: temperatures above 1420K present
  w.warn("O'Neill fO2: temperatures above 1420K present")
/Users/thomas/Desktop/MagmaPandas/src/MagmaPandas/geochemistry/fO2.py:163: UserWarning: O'Neill fO2: temperatures above 1420K present
  w.warn("O'Neill fO2: temperatures above 1420K present")
/Users/thomas/Desktop/MagmaPandas/src/MagmaPandas/geochemistry/fO2.py:163: UserWarning: O'Neill fO2: temperatures above 1420K present
  w.warn("O'Neill fO2: temperatures above 1420K present")
/Users/thomas/Desktop/MagmaPandas/src/MagmaPandas/geochemistry/fO2.py:163: UserWarning: O'Neill fO2: temperatures above 1420K present
  w.warn("O'Neill fO2: temperatures above 1420K present")
/Users/thomas/Desktop/MagmaPandas/src/Ma

[-0.03537500000000001,
 -0.04800000000000002,
 -0.05712500000000003,
 -0.01975,
 -0.026625000000000006,
 -0.04550000000000002,
 -0.02975000000000001,
 -0.039500000000000014,
 -0.07062500000000003,
 -0.03575000000000001,
 -0.018125,
 -0.04887500000000002,
 -0.03237500000000001,
 -0.028125000000000008,
 -0.038875000000000014,
 -0.03187500000000001,
 -0.040500000000000015,
 -0.03787500000000001,
 -0.05450000000000003,
 -0.041750000000000016]

In [7]:
F1

name
PI054-07-06   -0.035375
PI054-07-05   -0.048000
PI056-04-02   -0.057125
PI116-04-01   -0.019750
PI056-05-01   -0.026625
PI089-03-01   -0.045500
PI074-05-01   -0.029750
PI052-02-01   -0.039500
PI056-04-03   -0.070125
PI085-04-02   -0.035750
PI052-03-04   -0.018125
PI085-04-05   -0.048875
PI085-01-01   -0.032375
PI085-01-06   -0.028125
PI085-03-02   -0.038875
PI052-03-02   -0.031875
PI085-03-01   -0.040500
PI085-01-05   -0.037875
PI085-02-01   -0.054250
PI052-05-01   -0.041750
dtype: float64

In [ ]:
from MagmaPandas import Olivine

In [ ]:
Olivine({"MgO": 1.4, "FeO":0.6, "SiO2":1}, columns=melt.columns, index=melt.index)